In [ ]:
import random

# 读取train数据，划分验证集
train_index = []
train_label = []
valid_index = []
valid_label = []
with open('Lab5_Data/train.txt', 'r') as f:
    line  =  f.readline() 
    while line: 
      strs = line.split(',')
      if strs[0] != 'guid':
        if random.randint(0,9) < 8: 
            train_index.append(strs[0])
            train_label.append(strs[1].strip())
        else:
            valid_index.append(strs[0])
            valid_label.append(strs[1].strip())
      line = f.readline()

# 读取测试数据
test_index = []
with open('Lab5_Data/test_without_label.txt', 'r') as f:
    line  =  f.readline() 
    while line: 
      strs = line.split(',')
      if strs[0] != 'guid':
        test_index.append(strs[0])
      line = f.readline()

In [ ]:
print(train_index[:10])
print(train_label[:10])
print(valid_index[:10])
print(valid_label[:10])
print(test_index[:10])
print(len(test_index))

['4597', '26', '4383', '212', '2626', '3042', '4713', '2073', '2688', '3143']
['negative', 'neutral', 'negative', 'positive', 'positive', 'neutral', 'positive', 'negative', 'negative', 'positive']
['2020', '4956', '3624', '2855', '4784', '3244', '725', '138', '1562', '3888']
['positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive']
['8', '1576', '2320', '4912', '3821', '1306', '4555', '259', '3216', '881']
511


In [ ]:
# 将标签转为数字
label_to_index = {'negative':0, 'neutral':1, 'positive':2}
index_to_label = {0:'negative', 1:'neutral', 2:'positive'}

train_label_index = [label_to_index[item] for item in train_label]
valid_label_index = [label_to_index[item] for item in valid_label]
print(len(train_label_index))
print(train_label_index[:10])
print(len(valid_label_index))
print(valid_label_index[:10])

3206
[0, 1, 0, 2, 2, 1, 2, 0, 0, 2]
794
[2, 2, 2, 2, 1, 2, 2, 2, 2, 2]


In [ ]:
# 根据上面划分的id读取具体数据
train_text = []
for index in train_index:
    file = index + '.txt'
    with open('Lab5_Data/data/' + file, 'r', encoding='utf-8', errors='ignore') as f:
        line = f.readline() 
        train_text.append(line)
print(len(train_text))
print(train_text[:1])

valid_text = []
for index in valid_index:
    file = index + '.txt'
    with open('Lab5_Data/data/' + file, 'r', encoding='utf-8', errors='ignore') as f:
        line = f.readline() 
        valid_text.append(line)
print(len(valid_text))
print(valid_text[:1])

test_text = []
for index in test_index:
    file = index + '.txt'
    with open('Lab5_Data/data/' + file, 'r', encoding='utf-8', errors='ignore') as f:
        line = f.readline() 
        test_text.append(line)
print(len(test_text))
print(test_text[:1])

3206
['RT @AmitSwami77: The conspirators have an evil eye & are now set to physically attack Asaram Bapu Ji! #WeDemandSafety4Bapuji http://t.co/N8\n']
794
['# # #abandoned #дäƤˤȿꤿ #дäˤȿꤿ \n']
511
['Energetic training today with our San Antonio New Dollars/New Partners trainees \n']


In [ ]:
# 需要先将文本tokenize
import nltk
import string 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

punctuations = set(string.punctuation)
stopWords = set(stopwords.words('english'))

print(punctuations)

def tokenizer(str):
    words = word_tokenize(str)
    wordsFiltered = [word for word in words if word not in punctuations and word not in stopWords]
    return wordsFiltered

# 转为dict，便于写入jsonlines文件
def convert_dict_list(tokenizer, id, text, label=[], need_label=False):
    dict_list = []
    if need_label:
        assert len(text) == len(label)
        for i in range(0, len(text)):
            dict_list.append({'id': id[i], 'text': tokenizer(text[i]), 'label': label[i]})
    else:
        for i in range(0, len(text)):
            dict_list.append({'id': id[i], 'text': tokenizer(text[i])})
    return dict_list

train_dict_list = convert_dict_list(id=train_index, text=train_text, label=train_label_index, tokenizer=tokenizer, need_label=True)
valid_dict_list = convert_dict_list(id=valid_index, text=valid_text, label=valid_label_index, tokenizer=tokenizer, need_label=True)
test_dict_list = convert_dict_list(id=test_index, text=test_text, tokenizer=tokenizer)
print(train_dict_list[0])
print(valid_dict_list[0])
print(test_dict_list[0])

{'\\', ';', '.', '_', '"', '-', '>', '/', '!', '^', '&', '<', '@', '[', '=', '|', '~', ')', '%', ',', '*', '#', ':', '?', "'", '`', ']', '(', '{', '}', '+', '$'}


[nltk_data] Downloading package punkt to C:\Users\Guo
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Guo
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'id': '4597', 'text': ['RT', 'AmitSwami77', 'The', 'conspirators', 'evil', 'eye', 'set', 'physically', 'attack', 'Asaram', 'Bapu', 'Ji', 'WeDemandSafety4Bapuji', 'http', '//t.co/N8'], 'label': 0}
{'id': '2020', 'text': ['abandoned', 'дäƤˤȿꤿ', 'дäˤȿꤿ'], 'label': 2}
{'id': '8', 'text': ['Energetic', 'training', 'today', 'San', 'Antonio', 'New', 'Dollars/New', 'Partners', 'trainees']}


In [ ]:
# 观察文本中tokenize后最长的长度
print(len(train_dict_list))
print(len(valid_dict_list))
print(len(test_dict_list))
train_dict_list = [item for item in train_dict_list if len(item['text']) > 0]
valid_dict_list = [item for item in valid_dict_list if len(item['text']) > 0]
test_dict_list = [item for item in test_dict_list if len(item['text']) > 0]
print(len(train_dict_list))
print(len(valid_dict_list))
print(len(test_dict_list))

3206
794
511
3202
794
511


In [ ]:
import jsonlines

# 写入jsonlines文件
def save_to_jsonl(dict_list, filename):
    with jsonlines.open('jsonl/'+filename, mode='w') as writer:
        for line in dict_list:
            writer.write(line)

save_to_jsonl(train_dict_list, 'train.jsonl')
save_to_jsonl(valid_dict_list, 'valid.jsonl')
save_to_jsonl(test_dict_list, 'test.jsonl')

In [ ]:
train_dict_len = [len(item['text']) for item in train_dict_list]
valid_dict_len = [len(item['text']) for item in train_dict_list]
test_dict_len = [len(item['text']) for item in train_dict_list]

In [ ]:
print(max(train_dict_len))
print(max(valid_dict_len))
print(max(test_dict_len))

29
29
29


In [ ]:
print(min(train_dict_len))
print(min(valid_dict_len))
print(min(test_dict_len))

1
1
1
